In [ ]:
look up by index

# GOAL

1. find features
2. use features to predict response ratio

# IMPORTS

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper

# GET DATA

In [ ]:
path_users = "/Users/gandalf/Documents/data/data_users.pkl"
user_df = pd.read_pickle(path_users)
# print(user_df.columns)
# user_df.head(2)

# REMOVE NEW WORLD
print(len(user_df))
user_df = user_df[(user_df.longitude > -25) | (user_df.latitude < 0)]
user_df = user_df[user_df.longitude != 0]
print(len(user_df))

In [ ]:
from shapely.geometry import Point
from geopandas import GeoDataFrame
import geopandas as gpd

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

geometry = [Point(xy) for xy in zip(user_df.longitude, user_df.latitude)]
loc_df = user_df[['longitude', 'latitude']]
crs = {'init': 'epsg:4326'}
gdf = GeoDataFrame(loc_df, crs=crs, geometry=geometry)

fig, ax = plt.subplots(figsize = (16,10))

ax.set_aspect('equal')

world.plot(ax=ax, color='#66FF66', edgecolor='black')

gdf.plot(ax=ax, marker='o', color='#FF00FF', markersize=5)
ax.set_facecolor("#00FFFF")
plt.show();

In [ ]:
user_df.describe()

In [ ]:
# pick columns
cols_for_regression = ['has_about','len_about',
       'age', 'gender', 'inRelationship', 'isClean', 'isNight',
       'isStudent', 'latitude', 'longitude', 'maxCost',
       'minCost', 'petsOk', 'pictureId', 'sent']
X = user_df[cols_for_regression]
y = user_df['ratio']

# reformat columns
tf = {True:1, False:0, 1:1, 0:0}
mf = {'male':1, 'female':0, 1:1, 0:0}

X.has_about = X.has_about.map(tf)

X.gender = X.has_about.map(mf)
X = X.fillna(0)

# y_binary = (y > 0).map(tf)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=17)

mapper = DataFrameMapper([(X_train.columns, StandardScaler())])
scale = mapper.fit(X_train)
X_train = pd.DataFrame(mapper.transform(X_train), index=X_train.index, columns=X_train.columns)
X_test = pd.DataFrame(mapper.transform(X_test), index=X_test.index, columns=X_test.columns)

y_train_binary = (y_train > 0).map(tf)
y_test_binary = (y_test > 0).map(tf)

In [ ]:
print(y_test_binary.sum())
print(len(y_test_binary))

In [ ]:
print("baseline: {:.5}".format(1-(y_test_binary.sum()/len(y_test_binary))))

# LINEAR REGRESSION

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)
model.score(X_test, y_test)

# LOGISTIC REGRESSION

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train_binary)
model.score(X_test, y_test_binary)

# DECISION TREE

In [ ]:
dt = DecisionTree()
dt.fit(X_train, y_train)
predicted_y = dt.predict(X_test)